# Portfolio selection simulation

Simple portfolio simulation.

The prices follow geometric Brownian motions.

We follow a naive rebalancing strategy between three components:
 - stocks
 - bonds
 - commodity

In [ ]:
using ConcurrentSim
using Random
using ResumableFunctions
using Printf

## Structures

In [ ]:
# Asset types
mutable struct Asset
    name::String
    price::Float64
    volatility::Float64
    expected_return::Float64
end

# Portfolio manager
mutable struct Portfolio
    cash::Float64
    holdings::Dict{String, Float64}  # asset name => quantity
    total_value::Float64
    history::Vector{Tuple{Float64, Float64}}  # (time, value)
end

## Asset prices process

In [ ]:
# Market process: updates asset prices
@resumable function market_process(env::Environment, assets::Vector{Asset}, dt::Float64)
    rng = Xoshiro(42)
    while true
        @yield timeout(env, dt)
        
        # Update each asset price using geometric Brownian motion
        for asset in assets
            drift = asset.expected_return * dt
            shock = asset.volatility * sqrt(dt) * randn(rng)
            asset.price *= exp(drift + shock)
        end
        
        println(@sprintf("Time %.2f: Stock=\$%.2f, Bond=\$%.2f, Commodity=\$%.2f", 
                now(env), assets[1].price, assets[2].price, assets[3].price))
    end
end

## Rebalancing strategy

In [ ]:
# Rebalancing strategy: maintain target weights
function rebalance!(portfolio::Portfolio, assets::Vector{Asset}, target_weights::Dict{String, Float64})
    total_value = calculate_portfolio_value(portfolio, assets)
    
    # Sell all current holdings
    for asset in assets
        quantity = get(portfolio.holdings, asset.name, 0.0)
        portfolio.cash += quantity * asset.price
        portfolio.holdings[asset.name] = 0.0
    end
    
    # Buy according to target weights
    for asset in assets
        target_value = total_value * get(target_weights, asset.name, 0.0)
        quantity = target_value / asset.price
        portfolio.holdings[asset.name] = quantity
        portfolio.cash -= quantity * asset.price
    end
end

## Portfolio management

In [ ]:
# Initialize portfolio
function Portfolio(initial_cash::Float64)
    Portfolio(initial_cash, Dict{String, Float64}(), initial_cash, [(0.0, initial_cash)])
end

# Calculate portfolio value
function calculate_portfolio_value(portfolio::Portfolio, assets::Vector{Asset})
    total = portfolio.cash
    for asset in assets
        quantity = get(portfolio.holdings, asset.name, 0.0)
        total += quantity * asset.price
    end
    return total
end

# Portfolio manager process: periodically rebalances
@resumable function portfolio_manager(env::Environment, portfolio::Portfolio, 
                                     assets::Vector{Asset}, rebalance_interval::Float64)
    # Initial investment: equal weight
    target_weights = Dict("Stock" => 0.4, "Bond" => 0.4, "Commodity" => 0.2)
    
    while true
        current_time = now(env)
        
        # Rebalance portfolio
        rebalance!(portfolio, assets, target_weights)
        
        # Calculate new portfolio value
        portfolio.total_value = calculate_portfolio_value(portfolio, assets)
        push!(portfolio.history, (current_time, portfolio.total_value))
        
        println(@sprintf("Time %.2f: Rebalanced portfolio. Value: \$%.2f", 
                current_time, portfolio.total_value))
        println("  Holdings: Stock=$(portfolio.holdings["Stock"]), Bond=$(portfolio.holdings["Bond"]), Commodity=$(portfolio.holdings["Commodity"])")
        println("  Cash: \$$(round(portfolio.cash, digits=2))")
        
        # Adjust strategy based on portfolio performance
        if current_time > 0
            returns = (portfolio.total_value - portfolio.history[end-1][2]) / portfolio.history[end-1][2]
            if returns < -0.05  # If lost >5%, shift to bonds (risk-off)
                target_weights["Stock"] = 0.2
                target_weights["Bond"] = 0.6
                target_weights["Commodity"] = 0.2
                println("  → Risk-off mode: increasing bond allocation")
            elseif returns > 0.05  # If gained >5%, shift to stocks (risk-on)
                target_weights["Stock"] = 0.5
                target_weights["Bond"] = 0.3
                target_weights["Commodity"] = 0.2
                println("  → Risk-on mode: increasing stock allocation")
            end
        end
        
        @yield timeout(env, rebalance_interval)
    end
end

## Portfolio simulation

In [ ]:
# Run simulation
function run_portfolio_simulation()
    println("=== Dynamic Portfolio Selection Simulation ===\n")
    
    # Create simulation environment
    sim = Simulation()
    
    # Initialize assets
    assets = [
        Asset("Stock", 100.0, 0.25, 0.08),      # High return, high volatility
        Asset("Bond", 100.0, 0.10, 0.03),       # Low return, low volatility
        Asset("Commodity", 100.0, 0.30, 0.05)   # Medium return, very high volatility
    ]
    
    # Initialize portfolio with $10,000
    portfolio = Portfolio(10000.0)
    
    # Start processes
    @process market_process(sim, assets, 0.25)  # Update prices every 0.25 time units
    @process portfolio_manager(sim, portfolio, assets, 1.0)  # Rebalance every 1.0 time unit
    
    # Run simulation for 10 time units
    run(sim, 10.0)
    
    # Final results
    println("\n=== Final Results ===")
    final_value = calculate_portfolio_value(portfolio, assets)
    initial_value = portfolio.history[1][2]
    total_return = (final_value - initial_value) / initial_value * 100
    
    println(@sprintf("Initial Value: \$%.2f", initial_value))
    println(@sprintf("Final Value: \$%.2f", final_value))
    println(@sprintf("Total Return: %.2f%%", total_return))
end

In [ ]:
# Run the simulation
run_portfolio_simulation()